In [1]:
import wbdata
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
def inp_data():
    
    flag = None
    year = None
    country = None
    
    
    while True:
        #указываем 'country' для определения позиции и объема ВВП одной страны в заданом году, 
        #'top' - для вывода топ-10 стран по ВВП в заданном году
        flag = input('Type "country" or "top" or "stop" to cancel a request: ').strip()
        if flag == 'stop':
            print('Cancelling your request..')
            break
        elif flag == 'country' or flag == 'top':
            print('Got it!')
            break
        else:
            print('Sorry, I don\'t know such a command..')
        
    
    if flag == 'country' or flag == 'top':
        while True:
            year = input('Type a year from 1990 to 2019 or "stop" to cancel a request: ').strip()
            if year == 'stop':
                print('Cancelling your request..')
                break
            elif year.isdigit():
                if int(year) in range(1990, 2020):
                    print('Got it!')
                    break
                else:
                    print('Sorry, I can\'t understand.. Please, try again..')    
            else:
                print('Sorry, I can\'t understand.. Please, try again..')       
        
 
    if flag == 'country' and year != 'stop':
        while True:
            country = input('Type a country name or "stop" to cancel a request: ').strip()
            if country == 'stop':
                print('Cancelling your request..')
                break
            dic_countries = wbdata.search_countries(country)
            if len(dic_countries) == 0:
                print('Sorry, can\'t find this country..')
            elif len(dic_countries) > 1:
                if country == dic_countries[0]['name']:
                    country = dic_countries[0]['name']
                    print('Good choice! Let\'s check info about', country)
                    break   
                print('Find several countries, please choose one:')
                for n in dic_countries:
                    print(n['name'])                
            else:
                country = dic_countries[0]['name']
                print('Good choice! Let\'s check info about', country)
                break
    
    return flag, year, country

In [3]:
def info_getter(fl, yr, cnt=None):
    
    yr = int(yr)
    d_date = datetime(yr, 1, 1)
    df_gdp = wbdata.get_dataframe({'NY.GDP.MKTP.PP.CD':'GDP, PPP, $B'}, data_date=d_date)
    
    index_to_drop = df_gdp.index[:47]
    df_gdp_c = df_gdp.drop(index_to_drop)
    
    df_gdp_c = df_gdp_c.fillna({'GDP, PPP, $B':0})
    df_gdp_cs = df_gdp_c.sort_values(['GDP, PPP, $B'], ascending=False)
    
    df_gdp_cs['GDP, PPP, $B'] = round(df_gdp_cs['GDP, PPP, $B']/1000000000).apply(lambda x: int(x))
    df_gdp_cs['GDP, PPP, $B'] = df_gdp_cs['GDP, PPP, $B'].apply(lambda x: '{0:,}'.format(x).replace(',', ' '))
    
    if fl == 'country':
        ind = np.where(df_gdp_cs.index == cnt)
        pos = ind[0][0]+1
        gdp = df_gdp_cs.loc[cnt]['GDP, PPP, $B']
        print(cnt, '\nPosition in the World in ', yr, ': ', pos, '\nGPD, PPP: ', gdp, ' $B', sep='')
        
    if fl == 'top':
        print('Top ten countries in the World by GDP, PPP in ', yr, ':', sep='')
        print(df_gdp_cs.head(10).to_string(header=True))

In [5]:
flag, year, country = inp_data()
if flag!='stop' and year!='stop' and country!='stop':
    try:
        info_getter(flag, year, country)
    except:
        print('The request is cancelled.. for some unknown reason..')

Type "country" or "top" or "stop" to cancel a request: top
Got it!
Type a year from 1990 to 2019 or "stop" to cancel a request: 1990
Got it!
Top ten countries in the World by GDP, PPP in 1990:
                   GDP, PPP, $B
country                        
United States             5 963
Japan                     2 417
Germany                   1 543
Russian Federation        1 188
China                     1 115
Italy                     1 055
India                     1 049
France                    1 026
Brazil                      999
United Kingdom              963
